In [1]:
import sys
import warnings
warnings.filterwarnings(action='ignore')
import collections
from IPython.display import display
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import font_manager, rc
import pandas as pd
import mglearn
from sklearn.model_selection import train_test_split

mpl.rcParams['axes.unicode_minus'] = False
font_fname = '/Users/seongdae/Library/Fonts/NanumGothicLight.ttf'
font_name = font_manager.FontProperties(fname=font_fname).get_name()

rc('font', family=font_name)
# size, family
print ('font size : '+str(plt.rcParams['font.size']) )
print ('font family : '+str(plt.rcParams['font.family']) )
# import default setting
print('python version : {0}\npandas version : {1}\nmatplotlib version : {2}\nnumpy version : {3}\n'
.format(sys.version, pd.__version__, mpl.__version__, np.__version__))

font size : 10.0
font family : ['NanumGothic']
python version : 3.7.6 (v3.7.6:43364a7ae0, Dec 18 2019, 14:18:50) 
[Clang 6.0 (clang-600.0.57)]
pandas version : 0.25.3
matplotlib version : 3.1.2
numpy version : 1.18.1



# Decision Tree

In [6]:
# 결정 트리는 분류와 회귀 문제에서 널리 사용하는 모델
# 기본적으로 결정 트리는 결정에 다다르기 위해 예/아니오 질문을 이어 나가면서 학습, like 스무고개

In [7]:
# 결정 트리를 학습한다는 것은, 정답에 가장 빨리 도달하는 예/아니오 질문 목록을 학습하는 것
    # 머신러닝에서는 이런 질문들을 테스트라고함
    # 보통의 데이터들은 연속된 특성으로 구성됨, 연속적인 데이터에 적용할 테스트는
        # "특성 i는 값 å 보다 큰가?" 와 같은 형태로 이루어짐
# 순수 노드 : 데이터를 분할할때, 각 분할된 영역 (결정 트리의 리프_말단_터미널)이 한개의 타깃값(하나의 클래스, 하나의 회귀 분석 결과)
# 를 가질때를 가질 때까지 반복함. 타깃 하나로만 이뤄진 리프노드를 순수 노드(pure node)라고 함

In [8]:
# 일반적으로 트리를 짤때, 모든 리프 노드가 순수 노드가 될 떄까지 진행하면 모델이 매우 복잡해지고 훈련데이터에 과적합됨
# 순수 노드로 이루어진 트리는 훈련 세트에 100% 정확하게 맞는다는 의미
    # 과적합시에 트리에 의한 결정 경계들이 클래스 포인트들에게서 멀리 떨어진 이상치(outlier)하나에 너무 민감해짐
# 과적합을 막는 두가지 전략
    # 1. 트리 생성을 일찍 중단, 사전 가지치기(pre-pruning)
        # 트리의 최대 깊이, 리프의 최대 개수를 제한 or 노드가 분할하기 위한 포인트의 최소 개수를 지정
    # 2. 트리를 만든 후, 데이터 포인트가 적은 노드를 삭제하거나 병합하는 전략, 사후 가지치기(post-pruning) or 그냥 가지치기(pruning)라함

In [9]:
# scikit-learn 에선 DecisionTreeRegressor/DecisionTreeClassifier 구현
# 사전 가지치기만 지원

In [13]:
#  결정 트리 분류기
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, stratify = cancer.target, random_state = 42)
tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
print('훈련 세트 정확도 : {:.4f}'.format(tree.score(X_train, y_train)))
print('테스트 세트 정확도 : {:.4f}'.format(tree.score(X_test, y_test)))

훈련 세트 정확도 : 1.0000
테스트 세트 정확도 : 0.9371


In [35]:
# 예상했던 대로 사전 가지치기를 하지 않아서(==모든 리프노드가 순수노드), 훈련 세트 정확도는 1이나, 테스트 세트의 경우 선형 회귀보다 스코어가 낮음
# 개선을 위해서 매개변수를 줌
    # 트리 최대 깊이 제한 max_depth =4
tree_depth_pruning = DecisionTreeClassifier(random_state=0, max_depth=4).fit(X_train, y_train)
print('훈련 세트 정확도 : {:.4f}'.format(tree_depth_pruning.score(X_train, y_train)))
print('테스트 세트 정확도 : {:.4f}'.format(tree_depth_pruning.score(X_test, y_test)))
# 훈련 스코어는 낮아졌지만, 테스트에서 예전보다 높은 스코어를 얻음 == 과대적합을 억제

훈련 세트 정확도 : 0.9883
테스트 세트 정확도 : 0.9510


## 결정 트리 분석(시각화)

In [36]:
# tree modul의 exprot_graphviz 함수 사용, 트리 시각화
    # 이 함수는 그래프 저장용 텍스트 파일 포맷 .dot 파일 만듬
    # 매개변수 filled = True < 노드의 클래스가 구분되도록 색으로 칠해짐
    # 시각화를 위해 클래스 이븜과 특성 이름을 매개변수로 전달
from sklearn.tree import export_graphviz
export_graphviz(tree_depth_pruning, out_file='./dot_files/decision_tree.dot', class_names=['악성', '양성'],
               feature_names=cancer.feature_names, impurity= False, filled=True)
